In [16]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from itertools import combinations, product
import time
import ssl
import os
import re
from tqdm.notebook import tqdm
import random

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

In [2]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [17]:
def get_html_from_url(url):
  chrome_options = Options()
  chrome_options.add_argument("--headless")

  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options) 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  try:
    html = browser.page_source
  except:
    print('something wrong with ' + url)
    html = None
  return html

In [18]:
df_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\third_iteration\new_entries_for_extracting_3d_iter_questions.csv"
df_sentences = pd.read_csv(df_path)

In [19]:
df_sentences

,Unnamed: 0,0
0,0,top technology
1,1,next technology
2,2,upcoming technology
3,3,advanced technology
4,4,future technology
5,5,peak technology
6,6,revolutionary technology
7,7,engaging technology
8,8,technology will replace
9,9,technology will emerge


In [20]:
df_sentences.set_index(['Unnamed: 0'], inplace=True)

In [21]:
#df_sentences

In [22]:
#from first run --ignore
#df_sentences = df_sentences['new_list'].to_frame()
#df_sentences.dropna(axis=0, inplace=True)

In [23]:
search_link = 'https://www.quora.com/search?q='

#create search links
all_search_links = []
for i, row in tqdm(df_sentences.iterrows()):
    sentence_words = row.iloc[0].split()
    for i, word in enumerate(sentence_words):
        if '_' in word:
            word = word.replace('_', ' ')
            sentence_words[i] = word
    
    all_search_links.append(search_link+'%20'.join(sentence_words))


0it [00:00, ?it/s]

In [24]:
all_search_links

['https://www.quora.com/search?q=top%20technology',
 'https://www.quora.com/search?q=next%20technology',
 'https://www.quora.com/search?q=upcoming%20technology',
 'https://www.quora.com/search?q=advanced%20technology',
 'https://www.quora.com/search?q=future%20technology',
 'https://www.quora.com/search?q=peak%20technology',
 'https://www.quora.com/search?q=revolutionary%20technology',
 'https://www.quora.com/search?q=engaging%20technology',
 'https://www.quora.com/search?q=technology%20will%20replace',
 'https://www.quora.com/search?q=technology%20will%20emerge',
 'https://www.quora.com/search?q=technology%20will%20evolve',
 'https://www.quora.com/search?q=technology%20will%20develop',
 'https://www.quora.com/search?q=solar%20cryptocurrency',
 'https://www.quora.com/search?q=cryptocurrency%20crypto%20money%20market%20bitcoin%20currency%20cryptocurrencies',
 'https://www.quora.com/search?q=mining%20bitcoin%20miners%20hardware%20miner%20block%20cryptocurrency%20coins%20bitcoins%20reward

In [25]:
len(all_search_links)

38

In [26]:
class Extract_href_urls():
    def __init__(self, links):
        self.links = links
        self.all_box_links = []
        self.all_indexes = []
        self.get_question_links()
        self.all_hrefs = []        
        self.get_hrefs()
          
    def get_question_links(self):    
        for j, link in enumerate(self.links):
            url_html = get_html_from_url(link)
            if url_html is None:
                continue
            soup = BeautifulSoup(url_html)
            #find all boxes of groups
            self.all_box_l = soup.find_all('a', {'class':"q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 dxHfBI"})
            try:
                self.all_indexes.extend([df_sentences.index[j]]*len(self.all_box_l))
                self.all_box_links.extend(self.all_box_l)
            except:
                pass
            

    @staticmethod
    def get_url_tag(box):
        #By find_element not working in selenium using string parsing instead
        url=None
        box = str(box)
        split_box = box.split()
        for word in split_box:
            if 'href' in word:
                url = word.strip('href=').strip('"')
                break
        return url
    
    def get_hrefs(self): 
        for box in self.all_box_links:
            link_hrefs = self.get_url_tag(box)
            self.all_hrefs.append(link_hrefs)

In [27]:
res1 = Extract_href_urls(all_search_links)

C:\Users\Ravit\AppData\Local\Temp\ipykernel_11412\264970267.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver', options=chrome_options)


In [71]:
df_res = pd.DataFrame(res1.all_hrefs)

In [74]:
df_res['cluster_index'] = res1.all_indexes

In [76]:
len(df_res)

2467

In [79]:
df_res.drop_duplicates()

,0,cluster_index
0,https://www.quora.com/What-would-happen-to-cry...,-1
1,https://www.quora.com/What-solar-panels-would-...,-1
2,https://www.quora.com/Is-Cryptocurrency-not-ge...,-1
3,https://www.quora.com/Would-mining-cryptocurre...,-1
4,https://www.quora.com/If-I-have-free-electrici...,-1
...,...,...
2462,https://www.quora.com/How-can-a-politics-stude...,17
2463,https://www.quora.com/How-should-I-prepare-for...,17
2464,https://www.quora.com/If-Donald-Trump-is-not-r...,17
2465,https://www.quora.com/How-can-you-improve-your...,17


In [80]:
save_path = r'C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\second_iteration\links_for_second_quora_scrap.csv'
df_res.to_csv(save_path, index=False)

In [81]:
set(df_res['cluster_index'])

{-1, 0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 16, 17}

In [82]:
set(df_sentences.index) - set(df_res['cluster_index'])

{13, 15}